In [1]:
from mangaki.utils.data import Dataset

queryset = Rating.objects.all()  #.exclude(choice__in=['willsee', 'wontsee']) # Security if the dataset is public
triplets = queryset.values_list('user_id', 'work_id', 'choice')
titles = Work.objects.order_by('id').values_list('id', 'title')
categories = Work.objects.order_by('id').values_list('id', 'category__slug')
dataset = Dataset()

In [2]:
anonymized = dataset.make_anonymous_data(triplets, convert=str)

In [3]:
dataset.titles = dict(titles)
dataset.categories = dict(categories)

In [4]:
dataset.save_csv(suffix='-balse')

In [5]:
dataset.save('balse.pickle')